# Project: Wrangling and Analyze Data

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [1]:
# import packages
import pandas as pd
import requests as re
import os
import tweepy as tp
import json

In [3]:
# read WeRateDogs Twitter archive file on hand
twitter_archive = pd.read_csv(r'C:\Users\DE\Documents\Data wrangling_Udacity\twitter-archive-enhanced.csv')

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [4]:
# write file tsv containing images
path = os.getcwd() # get the current path
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = re.get(url)
if not os.path.exists(url.split('/')[-1]):
    with open(os.path.join(path, url.split('/')[-1]), mode = 'wb') as file:
        file.write(response.content)
        
# read image prediction tsv file into dataframe
image_prediction = pd.read_csv(r'C:\Users\DE\Documents\Data wrangling_Udacity\image-predictions.tsv', sep = '\t')

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [5]:
# query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# omit keys from submission to comply with Twitter's API terms and conditions
consumer_key = 'trwzMEOtXrAFiyJRW2hZVqA5c'
consumer_secret = 'JVI4rTTIyccfAABQDjprAGnpsQ1CEyIWaNfXSTfVei9V5mxVLv'
access_token = '1576941143874760704-dhdfW9p2fDUxO2bkxKGOBnTAa2mxZZ'
access_secret = 'IIuy2YQxQheg8zG3HN9C9czHAenjpn7rC7AALpctel1ov'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tp.API(auth, wait_on_rate_limit=True)

In [6]:
# avoid gather data again in case the file is ready
if not os.path.exists('tweet_json.txt'):
    # list twitter id into array to query Twitter API for additional information
    list_id = twitter_archive.tweet_id.values

    # store each tweet's entire set of JSON data in a file called tweet_json.txt file
    fails_dict = {}
    with open('tweet_json.txt', 'w', encoding = 'utf-8') as f: 
        for id in list_id: 
            try: 
                tweet = api.get_status(id, tweet_mode='extended')
                json.dump(tweet._json, f)
                f.write('\n')
            except tp.errors.TweepyException as e: 
                fails_dict[id] = e
                pass   

In [7]:
# read tweet_json.txt line by line into dataframe 
# with tweet ID, retweet count, favorite count
df_list = []
with open('tweet_json.txt', 'r', encoding = 'utf-8') as f:
    for count, line in enumerate(f): 
        tweet_id = json.loads(line)['id']
        retweet_count = json.loads(line)['retweet_count']
        favorite_count = json.loads(line)['favorite_count']
        df_list.append({'tweet_id': tweet_id, 
        'retweet_count': retweet_count, 
        'favorite_count': favorite_count})

tweet_api = pd.DataFrame(df_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])       

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### Quality Issues

1. Change datatype timestamp from string into datetime to exclude tweets beyond August 1st, 2017


In [8]:
# check datatype of timestamp column
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [9]:
# check data format
twitter_archive.timestamp

0       2017-08-01 16:23:56 +0000
1       2017-08-01 00:17:27 +0000
2       2017-07-31 00:18:03 +0000
3       2017-07-30 15:58:51 +0000
4       2017-07-29 16:00:24 +0000
                  ...            
2351    2015-11-16 00:24:50 +0000
2352    2015-11-16 00:04:52 +0000
2353    2015-11-15 23:21:54 +0000
2354    2015-11-15 23:05:30 +0000
2355    2015-11-15 22:32:08 +0000
Name: timestamp, Length: 2356, dtype: object

2. Rating denominator is equal to 0

In [10]:
twitter_archive[twitter_archive.rating_denominator == 0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None


3. Column expanded_urls has more 2 links, one in which is not twitter link or duplicated links

In [11]:
# randomly check the data format in expanded_urls
twitter_archive.expanded_urls.sample(10)

950     https://twitter.com/dog_rates/status/752173152...
805     https://twitter.com/dog_rates/status/772114945...
425     https://twitter.com/dog_rates/status/782305867...
1210    https://twitter.com/dog_rates/status/715680795...
1173    https://twitter.com/dog_rates/status/720340705...
592     https://twitter.com/dog_rates/status/798933969...
687     https://twitter.com/dog_rates/status/788039637...
277     https://twitter.com/dog_rates/status/840370681...
2288    https://twitter.com/dog_rates/status/667176164...
1335    https://twitter.com/dog_rates/status/705239209...
Name: expanded_urls, dtype: object

In [12]:
# check random data to see expanded_urls
twitter_archive.at[98, 'expanded_urls']

'https://www.gofundme.com/help-my-baby-sierra-get-better,https://twitter.com/dog_rates/status/873213775632977920/photo/1,https://twitter.com/dog_rates/status/873213775632977920/photo/1'

4. Column name contains wrong names, convert None into NaN

In [13]:
# special character is not the right name is: a, the, ...
twitter_archive.name.sample(10)

323          None
1248         None
1349         None
1042         None
1450         None
1783         None
360           Mia
1235        Sunny
817     Sebastian
1908       Bailey
Name: name, dtype: object

5. Tweet_ID = 740373189193256964 the dog has the rating is 14/10 

In [14]:
# check data based on url by visual assessment to see the rating
twitter_archive.at[twitter_archive[twitter_archive.tweet_id == 740373189193256964].index.values[0], 'expanded_urls']

'https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1'

6. Tweet_ID = 810984652412424192 the dog does not have the rating yet

In [15]:
# check data based on url by visual assessment to see the rating
# this tweet_id is not displayed -  24/7 not the rating
twitter_archive.at[twitter_archive[twitter_archive.tweet_id == 810984652412424192].index.values[0], 'expanded_urls']

'https://www.gofundme.com/sams-smile,https://twitter.com/dog_rates/status/810984652412424192/photo/1'

7. Tweet_ID = 722974582966214656 the dog has the rating is 13/10

In [16]:
# check data based on url by visual assessment to see the rating
twitter_archive.at[twitter_archive[twitter_archive.tweet_id == 722974582966214656].index.values[0], 'expanded_urls']

'https://twitter.com/dog_rates/status/722974582966214656/photo/1'

8. Tweet_ID = 666287406224695296 the dog has the rating is 9/10

In [17]:
# check data based on url by visual assessment to see the rating
twitter_archive.at[twitter_archive[twitter_archive.tweet_id == 666287406224695296].index.values[0], 'expanded_urls']

'https://twitter.com/dog_rates/status/666287406224695296/photo/1'

9. Tweet_ID = 775096608509886464 does not exist, it is a duplication of Tweet_ID = 740373189193256964

In [18]:
# the id in the url is 740373189193256964 not 775096608509886464
twitter_archive.at[twitter_archive[twitter_archive.tweet_id == 775096608509886464].index.values[0], 'expanded_urls']

'https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1'

### Tidiness Issues

1. Remove data in twitter-archive-enhancement.csv for matching with the image_prediction.tsv - only data not beyond August 1st, 2017

2. Match image prediction columns in image_prediction.tsv to define dog tweet_id and columns retweet count and favorite count in tweet_api for analysis

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [136]:
# Make copies of original pieces of data
twitter_archive_cp = twitter_archive.copy()
image_prediction_cp = image_prediction.copy()
tweet_api_cp = tweet_api.copy()

### Issue #1:

#### Define: Change datatype timestamp from string into datetime to exclude tweets beyond August 1st, 2017

#### Code

In [137]:
# change datatype
twitter_archive_cp.timestamp = pd.to_datetime(twitter_archive_cp.timestamp)

# exclude data with timestamp beyond August 1st, 2017
twitter_archive_cp = twitter_archive_cp.loc[twitter_archive_cp.timestamp <= '2017-08-01']


#### Test

In [138]:
# check datatype
twitter_archive_cp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2354 entries, 2 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2354 non-null   int64              
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2354 non-null   datetime64[ns, UTC]
 4   source                      2354 non-null   object             
 5   text                        2354 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2295 non-null   object             
 10  rating_numerator            2354 non-null   int64           

In [139]:
# check the data in datetime to make sure not beyond 2017, Aug 1st.
(twitter_archive_cp.timestamp > '2017-08-01').sum()

0

### Issue #2:

#### Define: Rating denominator is equal to 0
- Rating denominator should not be equal to 0, if exist, need to be excluded. 

#### Code

In [140]:
# exclude the record out of data
twitter_archive_cp = twitter_archive_cp[twitter_archive_cp.rating_denominator != 0]

#### Test

In [141]:
# test the result
(twitter_archive_cp.rating_denominator == 0).sum()

0

### Issue #3: 

#### Define: Column expanded_urls has more than 2 links, one in which is not twitter link or duplicated links
- Tweet id has duplicated urls 
- Tweet id has other link apart from Twitter link


#### Code

In [157]:
import re

In [244]:
# create a serie to check url different than twitter url
arr = twitter_archive_cp['expanded_urls'].apply(lambda x: len(set(x.split(','))) if str(x) != 'nan' else x)
# check the rows that number of tweet_id having more than 1 url
print('number of tweet_id having more than 1 url', arr[(arr > 1)].sum()) 
print('number of tweet_id having more than 2 urls', arr[(arr > 2)].sum())

# apply for tweet_id having duplicated urls
getOnelink = lambda x: list(set(x.split(',')))[0] if (str(x) != 'nan' and len(list(set(x.split(',')))) == 1) else x
twitter_archive_cp['expanded_urls'] = twitter_archive_cp['expanded_urls'].apply(lambda x: getOnelink(x))

# apply for tweet_id having more than 1 unique url
def getTwitter(a): 
    return list(filter(lambda x: re.findall(r'.+twitter.com.+', x), a))[0]

getTwitterlink = lambda x: getTwitter(x.split(',')) if (str(x) != 'nan' and len(list(set(x.split(',')))) > 1) else x
twitter_archive_cp['expanded_urls'] = twitter_archive_cp['expanded_urls'].apply(lambda x: getTwitterlink(x))


number of tweet_id having more than 1 url 82.0
number of tweet_id having more than 2 urls 0.0


#### Test

In [134]:
arr = twitter_archive_cp['expanded_urls'].apply(lambda x: len(x.split(',')) if str(x) != 'nan' else x)

(arr >1).sum()


0

### Issue #4: 

### Issue #5:

### Issue #6: 

### Issue #7:

### Issue #8:

### Issue #9:

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization